# Втор парцијален испит по Основи на Вештачката Интелигенција - 1/2

**Датум**: 18.01.2023

## 1. Амазон
Ја разгледуваме задачата за играта Амазон, од аудиториските вежби.

### Прашања и задачи

Бидејќи дрвото на играта е мало, во решението од аудиториските се разгранува целото дрво. Сакаме да дознаеме колку вкупно состојби се разгрануваат при пребарувањето низ дрвото.

1.1. **(5 поени)** Без користење на калкулатор, дискутирајте на кој начин ќе процените колку состојби треба да се разгранат за да се истражи целото дрво на играта.
    

* најдобро е да поставиме бројач на почетокот на кодот во минимакс алгоритмот кој ќе се инкрементира при секое разгранување
* бројачот треба да е глобална променлива за да може да се повика вон функцијата кога ќе сакаме да видиме колку јазли разгранил минимакс

1.2. **(5 поени) (код)** Пресметајте колку точно состојби се разгрануваат за да се истражи целото дрво на играта?

In [ ]:
def minimax(state, player):
    global counter
    counter+=1
    if check_score(state,player)!= 'keep_playing':
            return scores[check_score(state)],None 
    best = plus_inf if player == 'MIN' else minus_inf
    for next_state, next_move in expand_state(state, player):
        result = minimax(next_state, other_player(player))
        if player == 'MIN' and result <= best:
            best = result
        elif player == 'MAX' and result > best:
            best = result
    return best

1.3. **(5 поени)** Што е алфа-бета поткастрување? Зошто користиме алфа-бета поткастрување во оваа игра?

* алфа бета поткаструвањето всушност помага играта да се одвива побрзо затоа што нема да бидат разграснувани сите јазли
* тоа всушност однапред предвидува дали некој јазол ќе треба да се разграни, на пример ако на ред е да бира мин и има земено некоја вредност 4 а во другиот пар избор има прва гранка 2, што за МИН е секако помало од 4,но тоа за МАКС секако ќе биде неприфатливо и МАКС ќе избере 4 па воопшто нема ни да разгледува понатаму МИН

1.4. **(10 поени) (код)** Покажете дека користењето на постапката за алфа-бета поткастрување се исплати во оваа игра. На кој начин подобро ќе се покаже: (1) преку времето потребно за извршување на алгоритмите или (2) преку броење на разгранетите јазли?

In [ ]:
# prethodniot kod nema da go pishuvam tuku samo minimax funkcijata so alfa beta potkastruvanje so koja jas ja reshiv zadachata
#minimax so dlabochina
def minimax_alpha_beta(state, player,alpha=minus_inf, beta=plus_inf, depth=0):
    global counter
    counter+=1
    if check_score(state,player)!= 'keep_playing':
        return scores[check_score(state)],None 
    if depth == 3:
        return evaluate_state(state),None
    best = plus_inf if player == 'MIN' else minus_inf
    best_move = None
    symbol='x' if player=='MAX' else 'o'
    for next_state, next_move in expand_state(state, player):
        result, result_move = minimax(next_state, turn(player),alpha,beta,depth+1)
        if player == 'MIN':
            if result <= alpha:
                return result, best_move
            if result < beta:
                beta = result
            if result < best:
                best = result
                best_move = move
        elif player == 'MAX':
            if result >= beta:
                return result, best_move
            if result > alpha:
                alpha = result
            if result > best:
                best = result
                best_move = result_move
    return best, best_move

* поголема и поточна разлика ќе се види со бројот на разгранети јазли, а тоа се прави со повикување на глобален бројач кој инкрементира при секое повикување на минимакс

Целото дрво брзо се истражува на компјутер од 2022 година. Во продолжение разгледуваме случај кога овој алгоритам се поставува на компјутер кој нема доволно пресметковна моќ и мемориски ресурси за да го пребара целото дрво во логично време за еден потег. Ова е случај кога таблата наместо 3х3 ќе биде 6х6. Стандардната постапка во овој случај е да се ограничи длабочината до која пребарува алгоритамот, па тогаш да се проценува исходот од играта преку функција за евристика.

1.5. **(20 поени) (код)** Предложете функција за проценка на победникот (евристика) за оваа игра. Имплементирајте ја во код. Функцијата може да се напише без да ја тестирате со готовата игра од аудиториските вежби. Важно е да се објасни секоја логичка целина од кодот на функцијата која ќе ја напишете.

* ако до полето побрзо стига бела кралица, нејзино е, ако не на црната

In [3]:
def who_owns_square(state, square_position, N):
    """
    x,y ќе бидат координатите на позицијата
    N е големината на таблата
    алгоритмот работи така што ќе имаме една листа која ќе брои кој колку можност има да направи потег до една позиција
    оној со поголем број на потези губи, затоа што другиот е поблиску
    -1 враќа за црни, 1 за бели, 0 за нерешено
    """
    x, y = square_position
    number_of_moves = []
    for queen_color in ['white', 'black']:
        goal_state = move(state, queen_color,square_position)
        number_of_moves.append(a_star_search(state, goal_state, queen_color, N)[0])
        
    if number_of_moves[0] > number_of_moves[1]: # ako white pravi povekje moves za da stigne do istoto mesto
        return -1
    elif number_of_moves[0] < number_of_moves[1]:
        return 1
    else:
        return 0

In [4]:
def who_is_at_advantage(state, N):
    """
    имаме бројач total_score кој ако победува црната кралица ќе е негативен, а ако е позитивен белата
    ќе броиме победник така што гледаме за непополнетите места, кој е поблиску до него, има поголема 
    шанса да го освои тоа место преку горната функција
    """
    total_score = 0
    for row_index, row in enumerate(state):
        for col_index, element in enumerate(row):
            if element == '.':
                score = who_owns_square(state, (row_index, col_index), N)
                total_score += score # ke e <0 ako black ima povekje polinja, ke e >0 ako white ima povekje polinja
    return total_score